In [1]:
import ctypes
ctypes.windll.kernel32.SetThreadExecutionState(0x80000002)
# ctypes.windll.kernel32.SetThreadExecutionState(0x80000000)


-2147483648

In [ ]:
from stable_baselines3 import PPO, DQN
from sb3_contrib import TQC

In [4]:
import os
import grid2op
from stable_baselines3 import PPO, DQN
from sb3_contrib import TQC
from grid2op.Runner import Runner
from agent_wrapper import Grid2opAgentWrapper
# from env_wrapper import Grid2opEnvWrapper
from env_wrapper_custom import Grid2opEnvWrapper
from stable_baselines3.common.env_util import make_vec_env

from grid2op.Reward import RedispReward, L2RPNReward, EpisodeDurationReward


env_name = "l2rpn_case14_sandbox"
base_path = os.path.expanduser("C:/Users/henri/data_grid2op")
train_dir = os.path.join(base_path, f"{env_name}_train")
val_dir = os.path.join(base_path, f"{env_name}_val")
test_dir = os.path.join(base_path, f"{env_name}_test")

def directories_exist(train_path, val_path, test_path):
    return os.path.exists(train_path) and os.path.exists(val_path) and os.path.exists(test_path)

# Make environment
env = grid2op.make(env_name, reward_class=L2RPNReward)

if directories_exist(train_dir, val_dir, test_dir):
    print("Directories already exist. Skipping train-val-test split.")
else:
    print("Directories do not exist. Proceeding with train-val-test split.")
    # Perform the split
    try:
        nm_env_train, nm_env_val, nm_env_test = env.train_val_split_random(
            pct_val=10, # 10% validation
            pct_test=10, # 10% test
            add_for_train="train",
            add_for_val="val",
            add_for_test="test"
        )
        print(f"Training environment created: {nm_env_train}")
        print(f"Validation environment created: {nm_env_val}")
        print(f"Test environment created: {nm_env_test}")
    except OSError as e:
        print(f"An error occurred during splitting: {e}")


Directories do not exist. Proceeding with train-val-test split.
Training environment created: l2rpn_case14_sandbox_train
Validation environment created: l2rpn_case14_sandbox_val
Test environment created: l2rpn_case14_sandbox_test


In [ ]:
from stable_baselines3 import PPO, DQN, A2C
from sb3_contrib import ARS, QRDQN, TRPO, RecurrentPPO
from grid2op.Reward import RedispReward, L2RPNReward, EpisodeDurationReward
from stable_baselines3.common.env_util import make_vec_env

log_dir = "./tensorboard_logs_L2RPNReward/"

env_config = {
    # "backend_cls": ,
    # "backend_options": {},
    "env_name": "l2rpn_case14_sandbox", # "l2rpn_neurips_2020_track1_small"
    "env_is_test": False,
    # "obs_attr_to_keep": ["gen_p", "p_or" ,"load_p", "rho", "line_status"], # "gen_q", "gen_v",
    "act_type": "discrete", # "discrete" "box" "multi_discrete"
    # "act_attr_to_keep": ["change_line_status", "set_line_status_simple", "set_bus"], # set_line_status
    "reward_class": L2RPNReward, # EpisodeDurationReward,
    "data_set": "train", # for training data set train/val/test
}

# Create the environment
vec_env = make_vec_env(lambda: Grid2opEnvWrapper(env_config), n_envs=1)

# List of algorithms to train
# algorithms = [PPO, DQN, A2C, ARS, QRDQN, TRPO, RecurrentPPO]
algorithms = [RecurrentPPO]


# Loop over each algorithm
for algo in algorithms:
    # Define the model path and tensorboard log name dynamically
    model_name = algo.__name__
    tb_log_name = f"{model_name}_default_sandbox"
    model_path = f"./{model_name}_default_sandbox"

    # Initialize the algorithm with custom hyperparameters
    sb3_algo = algo(
        # "MlpPolicy",
        # "LinearPolicy",
        # "MlpLstmPolicy",
        vec_env,
        verbose=0,
        tensorboard_log=log_dir,
    )

    # Train the model
    sb3_algo.learn(total_timesteps=1_000_000, tb_log_name=tb_log_name)

    # Save the model
    sb3_algo.save(model_path)

    print(f"{model_name} model saved at {model_path}")


## Validation

In [ ]:
from agent_wrapper import Grid2opAgentWrapper
from grid2op.Runner import Runner

nb_episode_test = 10
seeds_test_env = tuple(range(nb_episode_test))  # Seeds for the environment
seeds_test_agent = tuple(range(nb_episode_test))  # Seeds for the agent
# ts_ep_test = tuple(range(nb_episode_test))  # Timestamps for the episodes

# uncomment those two line if the env_config is the correct for the tested agent
# test_env_config = env_config
# test_env_config["data_set"] = "val"


test_env_config = {
    # "backend_cls": ,
    # "backend_options": {},
    "env_name": "l2rpn_case14_sandbox", # "l2rpn_neurips_2020_track1_small"
    "env_is_test": False,
    # "obs_attr_to_keep": ["gen_p", "p_or" ,"load_p", "rho", "line_status"], # "gen_q", "gen_v",
    "act_type": "discrete", # "discrete" "box" "multi_discrete"
    # "act_attr_to_keep": ["change_line_status", "set_line_status_simple", "set_bus"], # set_line_status
    "reward_class": L2RPNReward,
    "data_set": "val", # for training data set train/val/test
}


# test_vec_env = make_vec_env(lambda: Grid2opEnvWrapper(test_env_config), n_envs=1)

test_env = Grid2opEnvWrapper(test_env_config)

sb3_algo_to_test = PPO.load(model_path, env=test_env)

my_agent = Grid2opAgentWrapper(test_env, sb3_algo_to_test)
runner = Runner(**test_env._g2op_env.get_params_for_runner(),
                agentClass=None,
                agentInstance=my_agent)

res = runner.run(nb_episode=nb_episode_test,
                 env_seeds=seeds_test_env,
                 agent_seeds=seeds_test_agent,
                 # episode_id=ts_ep_test,
                 add_detailed_output=True,
                 path_save=model_path+"_val"
                 )

In [ ]:
for ep_data in res:
    chron_name, cum_reward, nb_time_step, max_ts, _, ep_data = ep_data
    print(f"Episode {i + 1}:")
    print(f"  Chronics Name: {chron_name}")
    print(f"  Cumulative Reward: {cum_reward}")
    print(f"  Number of Time Steps: {nb_time_step}")
    print(f"  Maximum Time Steps: {max_ts}")
    print("  Detailed Data:")
    for t, obs in enumerate(ep_data.observations):
        print(f"    Time Step {t}:")
        print(f"      Observation: {obs}")
        print(f"      Action Taken: {ep_data.actions[t]}")
        print(f"      Reward: {ep_data.rewards[t]}")
        break
    break

In [ ]:
import tensorflow as tf

log_dir = r"tensorboard_logs/PPO_default__sandbox_1"

data = {}

for event in tf.compat.v1.train.summary_iterator(log_dir):
    for value in event.summary.value:
        if value.tag not in data:
            data[value.tag] = {"steps": [], "values": []}
        data[value.tag]["steps"].append(event.step)
        data[value.tag]["values"].append(value.simple_value)

In [ ]:
from stable_baselines3 import PPO, DQN, A2C
from grid2op.Runner import Runner
from agent_wrapper import Grid2opAgentWrapper
from grid2op.Reward import EpisodeDurationReward

# Number of test episodes and seeds
nb_episode_test = 10
seeds_test_env = tuple(range(nb_episode_test))  # Environment seeds
seeds_test_agent = tuple(range(nb_episode_test))  # Agent seeds

# Validation environment configuration
test_env_config = {
    "env_name": "l2rpn_case14_sandbox",  # Change as needed
    "env_is_test": False,
    "act_type": "discrete",  # "discrete", "box", "multi_discrete"
    "reward_class": EpisodeDurationReward,
    "data_set": "val",  # Uses validation data set
}

algorithms = [PPO, DQN, A2C, ARS, QRDQN, TRPO]

# Loop through each algorithm
for algo in algorithms:
    # Define paths for loading models and saving validation results
    model_name = algo.__name__
    model_path = f"./{model_name}_default_sandbox"
    validation_path = f"{model_path}_val"

    # Create the test environment
    test_env = Grid2opEnvWrapper(test_env_config)

    # Load the trained model
    sb3_algo_to_test = algo.load(model_path, env=test_env)

    # Wrap the agent
    my_agent = Grid2opAgentWrapper(test_env, sb3_algo_to_test)

    # Initialize the Runner
    runner = Runner(**test_env._g2op_env.get_params_for_runner(),
                    agentClass=None,
                    agentInstance=my_agent)

    # Run validation
    print(f"Running validation for {model_name}...")
    res = runner.run(nb_episode=nb_episode_test,
                     env_seeds=seeds_test_env,
                     agent_seeds=seeds_test_agent,
                     add_detailed_output=True,
                     path_save=validation_path)

    # Save or log validation results
    print(f"Validation results saved at {validation_path}")
